In [80]:
from google.colab import files
import pandas as pd

In [ ]:
cust = files.upload() #google colab에서 사용하는 파일 업로드 방식임. 타 에디터 이용시 아래 주석과 같은 경로 지정 필요
#claim = pd.read_csv("../data/CLAIM.csv", encoding = "UTF-8", sep = ",")

Saving extracted_cust.csv to extracted_cust.csv


In [81]:
custdf = pd.read_csv('extracted_cust.csv', sep = ',', encoding = 'utf-8', error_bad_lines=False, engine = 'python', header = None, names = ['CUST_ID', 'DIVIDED_SET', 'SIU_CUST_YN', 'TOTALPREM', 'MINCRDT', 'JPBASE_HSHD_INCM'])#각 컬럼별 네이밍

In [82]:
custdf = custdf.drop(custdf.index[0])#첫 row는 컬럼명이 들어간 더미데이터이므로 삭제

In [83]:
custdf.isnull().sum()#결측치 확인

CUST_ID                0
DIVIDED_SET            0
SIU_CUST_YN         1793
TOTALPREM           5791
MINCRDT             9476
JPBASE_HSHD_INCM     680
dtype: int64

In [84]:
#신용도 미확인 값은 6으로 대체(데이터 정의에 따름)
custdf['MINCRDT'].fillna(value = 6, inplace = True)

In [85]:
custdf.isnull().sum()

CUST_ID                0
DIVIDED_SET            0
SIU_CUST_YN         1793
TOTALPREM           5791
MINCRDT                0
JPBASE_HSHD_INCM     680
dtype: int64

In [86]:
#신용도가 10보다 높으면 이상치이므로 행 삭제, 그러나 min값을 뽑아왔으므로 사례 없을 것으로 추정
custdf = custdf[custdf.MINCRDT <= 10]

In [87]:
#Total PREM 1000만단위 라벨링 (ex)176만 -> 0, 1700만 -> 1, 3억 -> 30 ...), 분석시 상관관계 낮게 나오면 범위 더 크게 할 것
custdf.dropna(subset = ['TOTALPREM'], inplace = True)
custdf['TOTALPREM'] = (custdf['TOTALPREM']/10000000).astype(int)

In [88]:
custdf.isnull().sum()#결측치 확인 2

CUST_ID                0
DIVIDED_SET            0
SIU_CUST_YN         1323
TOTALPREM              0
MINCRDT                0
JPBASE_HSHD_INCM     506
dtype: int64

In [89]:
custdf.dropna(subset = ['JPBASE_HSHD_INCM'], inplace = True)#결측치 삭제

In [90]:
#결측치 제거 완료
custdf.isnull().sum()

CUST_ID                0
DIVIDED_SET            0
SIU_CUST_YN         1275
TOTALPREM              0
MINCRDT                0
JPBASE_HSHD_INCM       0
dtype: int64

In [ ]:
claim = files.upload()#구글 코랩 파일 업로드 방식

Saving extracted_claim.csv to extracted_claim.csv


In [305]:
claimdf = pd.read_csv('extracted_claim.csv', sep = ',', encoding = 'utf-8', error_bad_lines=False, engine = 'python', header = None, names = ['CUST_ID', 'RESL_CD1', 'ACCI_OCCP_GRP', 'CAUS_CODE',  'DMND_RESN_CODE', 'VLID_HOSP_OTDA', 'HOSP_CODE', 'HOSP_SPEC_DVSN', 'CHME_LICE_NO', 'DMND_AMT', 'PAYM_AMT', 'NON_PAY_RATIO', 'HEED_HOSP_YN'])#13개 컬럼에 대한 네이밍

In [306]:
claimdf = claimdf.drop(claimdf.index[0])#역시 컬럼명으로 적혀있는 첫번째 row에 대한 삭제

In [307]:
claimdf.isnull().sum()#결측치에 대한 확인

CUST_ID              0
RESL_CD1             0
ACCI_OCCP_GRP     1417
CAUS_CODE            0
DMND_RESN_CODE       0
VLID_HOSP_OTDA       0
HOSP_CODE            9
HOSP_SPEC_DVSN       9
CHME_LICE_NO         0
DMND_AMT             0
PAYM_AMT             0
NON_PAY_RATIO        0
HEED_HOSP_YN         0
dtype: int64

In [308]:
#-------------- 직업별 정수형 라벨링 ---------------------------
# 결측치 포함 행 삭제
#사무직 1
#자영업 2
#주부 3
#교사 4
#예체능계 종사자 5
#운전직 6
#1차산업 종사자 7
#2차산업 종사자 8
#3차산업 종사자 9
#공무원 10
#고위 공무원 11
#단순 노무직 12
#기업/단체 임원 13
#고소득의료직 14
#전문직 15
#종교인/역술인 16
#대학교수/강사 17
#단순 사무직 18
#교육관련직 19
#학자/연구직 20
#학생 21
#법무직 종사자 22
#고소득 전문직 23
#의료직 종사자 24
#기타 25

#직업 추출에 사용된 출력문
#개선된 변수 확인 알고리즘 아이디어 있으신 분은 알려주시길(혹은 액셀 함수라도)
#for i in custDF['OCCP_GRP']: 
  #if i != '의료직 종사자' and i != '고소득 전문직' and i != '사무직' and i != None and i!='자영업' and i!='주부' and i!='교사' and i!='예체능계 종사자' and i!='운전직' and i!='1차산업 종사자' and i!='2차산업 종사자' and i!='3차산업 종사자' and i!='공무원' and i!='고위 공무원' and i!='단순 노무직' and i!='기업/단체 임원' and i!='고소득의료직' and i!='전문직' and i!='종교인/역술인' and i!='대학교수/강사' and i!='단순 사무직' and i!='교육관련직' and i!='학자/연구직' and i!='학생' and i!='법무직 종사자' and i!='기타' :
    #print(i)

#위 주석과 대응하는 정수형 라벨링(이 단계에서는 type casting 아직 안함)
claimdf['ACCI_OCCP_GRP'].replace('사무직', 1, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('자영업', 2, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('주부', 3, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('교사', 4, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('예체능계 종사자', 5, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('운전직', 6, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('1차산업 종사자', 7, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('2차산업 종사자', 8, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('3차산업 종사자', 9, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('공무원', 10, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('고위 공무원', 11, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('단순 노무직', 12, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('기업/단체 임원', 13, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('고소득의료직', 14, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('전문직', 15, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('종교인/역술인', 16, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('대학교수/강사', 17, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('단순 사무직', 18, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('교육관련직', 19, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('학자/연구직', 20, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('학생', 21, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('법무직 종사자', 22, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('고소득 전문직', 23, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('의료직 종사자', 24, inplace = True)

claimdf['ACCI_OCCP_GRP'].replace('기타', 25, inplace = True)



In [309]:
#병명 코드 각 첫번째 철자로 그룹화하기 위해 숫자 잘라낼 것
#정규표현식 이용 숫자 제거
claimdf['CAUS_CODE'] = claimdf['CAUS_CODE'].str.replace('\d+', '')


In [310]:
claimdf.dropna(subset = ['ACCI_OCCP_GRP'], inplace = True)#라벨링 후 직업 코드에 대한 결측치 삭제

In [311]:
claimdf.dropna(inplace = True)#클레임 데이터 내부 모든 결측치 제거

In [312]:
claimdf.isnull().sum()

CUST_ID           0
RESL_CD1          0
ACCI_OCCP_GRP     0
CAUS_CODE         0
DMND_RESN_CODE    0
VLID_HOSP_OTDA    0
HOSP_CODE         0
HOSP_SPEC_DVSN    0
CHME_LICE_NO      0
DMND_AMT          0
PAYM_AMT          0
NON_PAY_RATIO     0
HEED_HOSP_YN      0
dtype: int64

In [313]:
claimdf['CAUS_CODE_COUNT'] = claimdf.groupby(claimdf['CUST_ID'])["CAUS_CODE"].transform('nunique')#위와 동일

claimdf.drop(['CAUS_CODE'], axis=1, inplace = True)

In [314]:
claimdf['DMND_RESN_CODE_COUNT'] = claimdf.groupby(claimdf['CUST_ID'])["DMND_RESN_CODE"].transform('nunique')#위와 동일

claimdf.drop(['DMND_RESN_CODE'], axis=1, inplace = True)

In [315]:
claimdf['RESL_CD1_COUNT'] = claimdf.groupby(claimdf['CUST_ID'])["RESL_CD1"].transform('nunique')#위와 동일
claimdf.drop(['RESL_CD1'], axis=1, inplace = True)

In [316]:
# 라벨링
claimdf['HEED_HOSP_YN'] = claimdf['HEED_HOSP_YN'].replace(["N"],0)

claimdf['HEED_HOSP_YN'] = claimdf['HEED_HOSP_YN'].replace(["Y"],1)

HEED_HOSP_YN = claimdf.groupby('CUST_ID')['HEED_HOSP_YN'].count()
Y = claimdf.groupby("CUST_ID")["HEED_HOSP_YN"].sum()
HEED_HOSP_YN_rate = Y / HEED_HOSP_YN
HEED_HOSP_YN_rate

claim_DF= pd.DataFrame()
claim_DF["HEED_HOSP_YN_RATE"] = HEED_HOSP_YN_rate

In [317]:
claimdf.dtypes

CUST_ID                  object
ACCI_OCCP_GRP           float64
VLID_HOSP_OTDA          float64
HOSP_CODE               float64
HOSP_SPEC_DVSN          float64
CHME_LICE_NO            float64
DMND_AMT                float64
PAYM_AMT                float64
NON_PAY_RATIO           float64
HEED_HOSP_YN              int64
CAUS_CODE_COUNT           int64
DMND_RESN_CODE_COUNT      int64
RESL_CD1_COUNT            int64
dtype: object

In [318]:
claimdf["DMND_AMT_SUM"] = claimdf.groupby('CUST_ID')['DMND_AMT'].transform('sum')
claimdf["PAYM_AMT_SUM"] = claimdf.groupby('CUST_ID')['PAYM_AMT'].transform('sum')
claimdf["NON_PAY_RATIO_SUM"] = claimdf.groupby('CUST_ID')['NON_PAY_RATIO'].transform('sum')



In [319]:
claimdf

,CUST_ID,ACCI_OCCP_GRP,VLID_HOSP_OTDA,HOSP_CODE,HOSP_SPEC_DVSN,CHME_LICE_NO,DMND_AMT,PAYM_AMT,NON_PAY_RATIO,HEED_HOSP_YN,CAUS_CODE_COUNT,DMND_RESN_CODE_COUNT,RESL_CD1_COUNT,DMND_AMT_SUM,PAYM_AMT_SUM,NON_PAY_RATIO_SUM
1,5936,21.0,2.0,12537.0,95.0,11606.0,20000.0,20000.0,0.0,0,1,1,2,180000.0,180000.0,0.0
2,5936,21.0,2.0,12537.0,95.0,11606.0,30000.0,30000.0,0.0,0,1,1,2,180000.0,180000.0,0.0
3,5936,21.0,2.0,12537.0,95.0,11606.0,40000.0,40000.0,0.0,0,1,1,2,180000.0,180000.0,0.0
4,1043,3.0,6.0,12537.0,95.0,17647.0,60000.0,60000.0,0.0,0,4,1,6,3460000.0,3460000.0,0.0
5,8545,3.0,0.0,2305.0,30.0,25697.0,200000.0,200000.0,0.0,0,2,1,2,400000.0,400000.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119011,4355,21.0,1.0,10361.0,20.0,18691.0,7923.0,7500.0,1.0,0,1,1,1,61373.0,60950.0,2.0
119015,14068,10.0,4.0,10387.0,30.0,15727.0,6777.0,6455.0,1.0,0,3,3,6,1781297.0,1780975.0,6.0
119016,15545,1.0,4.0,7577.0,20.0,6903.0,23666.0,22500.0,1.0,0,2,1,2,305472.0,303525.0,7.0
119019,15545,1.0,1.0,7577.0,20.0,6903.0,7743.0,7500.0,1.0,0,2,1,2,305472.0,303525.0,7.0


In [320]:
claimdf.isnull().sum()#결측치 완전 제거 확인

CUST_ID                 0
ACCI_OCCP_GRP           0
VLID_HOSP_OTDA          0
HOSP_CODE               0
HOSP_SPEC_DVSN          0
CHME_LICE_NO            0
DMND_AMT                0
PAYM_AMT                0
NON_PAY_RATIO           0
HEED_HOSP_YN            0
CAUS_CODE_COUNT         0
DMND_RESN_CODE_COUNT    0
RESL_CD1_COUNT          0
DMND_AMT_SUM            0
PAYM_AMT_SUM            0
NON_PAY_RATIO_SUM       0
dtype: int64

In [321]:
claimdf = claimdf[(claimdf.VLID_HOSP_OTDA).astype(int) < 300]#유효 통원, 입원 일자가 10000이 넘어가는 이상치를 발견함. 유효입원일자가 높은 경우 실제로 아프거나 오류 데이터일 것이라고 생각하고 배제(이상치 제거 코드)

In [322]:
claimdf['CLAIM_CNT'] = claimdf['CUST_ID'].groupby(claimdf['CUST_ID']).transform('count')#group화하여 row수를 세주고, 그에 따른 카운트를 새 컬럼으로 만들어 추가, cust_id는 병합을 위해 남겨놓아야함을 인지

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [323]:
claimdf = claimdf.astype({'VLID_HOSP_OTDA': 'int'})#첫 row가 컬럼명으로 들어갔었기에 오브젝트형 타입으로 선언되어있었음. integer형으로 먼저 바꿔주는 작업 필요

In [324]:
claimdf.dtypes#타입 확인

CUST_ID                  object
ACCI_OCCP_GRP           float64
VLID_HOSP_OTDA            int64
HOSP_CODE               float64
HOSP_SPEC_DVSN          float64
CHME_LICE_NO            float64
DMND_AMT                float64
PAYM_AMT                float64
NON_PAY_RATIO           float64
HEED_HOSP_YN              int64
CAUS_CODE_COUNT           int64
DMND_RESN_CODE_COUNT      int64
RESL_CD1_COUNT            int64
DMND_AMT_SUM            float64
PAYM_AMT_SUM            float64
NON_PAY_RATIO_SUM       float64
CLAIM_CNT                 int64
dtype: object

In [325]:
claimdf['TOTAL_VLID_HOSP_OTDA'] = claimdf.groupby(claimdf['CUST_ID'])['VLID_HOSP_OTDA'].transform('sum')#cust_id 기준 vlid_hosp_otda의 합을 새 컬럼으로 만들고, 기존 컬럼 삭제
claimdf.drop(['VLID_HOSP_OTDA'], axis=1, inplace = True)

In [326]:
claimdf.isnull().sum()

CUST_ID                 0
ACCI_OCCP_GRP           0
HOSP_CODE               0
HOSP_SPEC_DVSN          0
CHME_LICE_NO            0
DMND_AMT                0
PAYM_AMT                0
NON_PAY_RATIO           0
HEED_HOSP_YN            0
CAUS_CODE_COUNT         0
DMND_RESN_CODE_COUNT    0
RESL_CD1_COUNT          0
DMND_AMT_SUM            0
PAYM_AMT_SUM            0
NON_PAY_RATIO_SUM       0
CLAIM_CNT               0
TOTAL_VLID_HOSP_OTDA    0
dtype: int64

In [327]:
claimdf['HOSP_VARIES'] = claimdf.groupby(claimdf['CUST_ID'])["HOSP_CODE"].transform('nunique')#nunique는 고유값을 세주는 카운트함수임. 즉 중복값을 제외하고 각자 다른 값들의 수를 세어 병원의 가짓수를 세서 컬럼화, 기존 컬럼 삭제
claimdf.drop(['HOSP_CODE'], axis=1, inplace = True)

In [328]:
claimdf['HOSP_DVSN_VARIES'] = claimdf.groupby(claimdf['CUST_ID'])["HOSP_SPEC_DVSN"].transform('nunique')#위와 동일
claimdf.drop(['HOSP_SPEC_DVSN'], axis=1, inplace = True)

In [329]:
claimdf['CHME_LICE_COUNT'] = claimdf.groupby(claimdf['CUST_ID'])["CHME_LICE_NO"].transform('nunique')#위와 동일
claimdf.drop_duplicates(['CUST_ID'], inplace = True)

In [330]:
claimdf

,CUST_ID,ACCI_OCCP_GRP,CHME_LICE_NO,DMND_AMT,PAYM_AMT,NON_PAY_RATIO,HEED_HOSP_YN,CAUS_CODE_COUNT,DMND_RESN_CODE_COUNT,RESL_CD1_COUNT,DMND_AMT_SUM,PAYM_AMT_SUM,NON_PAY_RATIO_SUM,CLAIM_CNT,TOTAL_VLID_HOSP_OTDA,HOSP_VARIES,HOSP_DVSN_VARIES,CHME_LICE_COUNT
1,5936,21.0,11606.0,20000.0,20000.0,0.0,0,1,1,2,180000.0,180000.0,0.0,6,12,1,1,2
4,1043,3.0,17647.0,60000.0,60000.0,0.0,0,4,1,6,3460000.0,3460000.0,0.0,10,111,5,5,7
5,8545,3.0,25697.0,200000.0,200000.0,0.0,0,2,1,2,400000.0,400000.0,0.0,2,0,2,2,2
6,4734,21.0,5503.0,30000.0,30000.0,0.0,0,3,3,4,410000.0,410000.0,0.0,6,21,2,1,3
7,9416,2.0,12126.0,300000.0,300000.0,0.0,0,5,3,10,10693767.0,10690000.0,0.0,18,259,6,4,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118903,15203,3.0,9182.0,125613.0,125613.0,0.0,0,1,2,1,159513.0,159513.0,0.0,2,4,1,1,1
118906,16048,1.0,14029.0,80000.0,80000.0,0.0,0,1,2,1,300000.0,300000.0,0.0,2,8,1,1,1
118916,2924,2.0,8671.0,90000.0,90000.0,0.0,0,1,1,1,90000.0,90000.0,0.0,1,12,1,1,1
118930,6451,9.0,14317.0,80000.0,80000.0,0.0,0,1,3,1,420000.0,420000.0,0.0,3,6,1,1,1


In [331]:
merged_claim_data = pd.merge(custdf, claimdf, how='right')#cust_id를 기준으로 병합될 것임

In [332]:
merged_claim_data.dropna(inplace= True)#병합된 데이터에서 결측치를 제거해줌

In [333]:
merged_claim_data.isnull().sum()#최종 데이터 결측치 확인

CUST_ID                 0
DIVIDED_SET             0
SIU_CUST_YN             0
TOTALPREM               0
MINCRDT                 0
JPBASE_HSHD_INCM        0
ACCI_OCCP_GRP           0
CHME_LICE_NO            0
DMND_AMT                0
PAYM_AMT                0
NON_PAY_RATIO           0
HEED_HOSP_YN            0
CAUS_CODE_COUNT         0
DMND_RESN_CODE_COUNT    0
RESL_CD1_COUNT          0
DMND_AMT_SUM            0
PAYM_AMT_SUM            0
NON_PAY_RATIO_SUM       0
CLAIM_CNT               0
TOTAL_VLID_HOSP_OTDA    0
HOSP_VARIES             0
HOSP_DVSN_VARIES        0
CHME_LICE_COUNT         0
dtype: int64

In [334]:
#merged_data = merged_claim_data.join(claim_DF, how ='natural')
merged_data = pd.merge(left = merged_claim_data , right = claim_DF, how = "inner", on = "CUST_ID")

In [335]:
merged_data

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,TOTALPREM,MINCRDT,JPBASE_HSHD_INCM,ACCI_OCCP_GRP,CHME_LICE_NO,DMND_AMT,PAYM_AMT,NON_PAY_RATIO,HEED_HOSP_YN,CAUS_CODE_COUNT,DMND_RESN_CODE_COUNT,RESL_CD1_COUNT,DMND_AMT_SUM,PAYM_AMT_SUM,NON_PAY_RATIO_SUM,CLAIM_CNT,TOTAL_VLID_HOSP_OTDA,HOSP_VARIES,HOSP_DVSN_VARIES,CHME_LICE_COUNT,HEED_HOSP_YN_RATE
0,5936,1.0,N,0.0,6.0,9872.0,21.0,11606.0,20000.0,20000.0,0.0,0,1,1,2,180000.0,180000.0,0.0,6,12,1,1,2,0.0
1,1043,1.0,N,2.0,6.0,4760.0,3.0,17647.0,60000.0,60000.0,0.0,0,4,1,6,3460000.0,3460000.0,0.0,10,111,5,5,7,0.0
2,8545,1.0,N,2.0,6.0,5070.0,3.0,25697.0,200000.0,200000.0,0.0,0,2,1,2,400000.0,400000.0,0.0,2,0,2,2,2,0.0
3,20267,1.0,Y,7.0,6.0,8951.0,3.0,25700.0,1600000.0,1600000.0,0.0,0,5,2,10,74755070.0,74734000.0,0.0,43,805,15,5,23,0.0
4,2778,1.0,N,0.0,6.0,4680.0,8.0,12423.0,260000.0,260000.0,0.0,0,4,3,7,6070000.0,6070000.0,0.0,10,144,6,2,6,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14672,14012,1.0,N,1.0,6.0,4284.0,6.0,12696.0,330000.0,330000.0,0.0,0,1,2,1,15330000.0,15330000.0,0.0,2,9,1,1,1,0.0
14673,15203,1.0,N,5.0,6.0,8510.0,3.0,9182.0,125613.0,125613.0,0.0,0,1,2,1,159513.0,159513.0,0.0,2,4,1,1,1,0.0
14674,16048,1.0,N,1.0,1.0,4016.0,1.0,14029.0,80000.0,80000.0,0.0,0,1,2,1,300000.0,300000.0,0.0,2,8,1,1,1,0.0
14675,2924,1.0,N,14.0,6.0,7228.0,2.0,8671.0,90000.0,90000.0,0.0,0,1,1,1,90000.0,90000.0,0.0,1,12,1,1,1,0.0


In [336]:
merged_data.dropna(inplace= True)#병합된 데이터에서 결측치를 제거해줌
merged_data.drop_duplicates(['CUST_ID'], inplace = True)#w중복값 제거

In [337]:
merged_data

,CUST_ID,DIVIDED_SET,SIU_CUST_YN,TOTALPREM,MINCRDT,JPBASE_HSHD_INCM,ACCI_OCCP_GRP,CHME_LICE_NO,DMND_AMT,PAYM_AMT,NON_PAY_RATIO,HEED_HOSP_YN,CAUS_CODE_COUNT,DMND_RESN_CODE_COUNT,RESL_CD1_COUNT,DMND_AMT_SUM,PAYM_AMT_SUM,NON_PAY_RATIO_SUM,CLAIM_CNT,TOTAL_VLID_HOSP_OTDA,HOSP_VARIES,HOSP_DVSN_VARIES,CHME_LICE_COUNT,HEED_HOSP_YN_RATE
0,5936,1.0,N,0.0,6.0,9872.0,21.0,11606.0,20000.0,20000.0,0.0,0,1,1,2,180000.0,180000.0,0.0,6,12,1,1,2,0.0
1,1043,1.0,N,2.0,6.0,4760.0,3.0,17647.0,60000.0,60000.0,0.0,0,4,1,6,3460000.0,3460000.0,0.0,10,111,5,5,7,0.0
2,8545,1.0,N,2.0,6.0,5070.0,3.0,25697.0,200000.0,200000.0,0.0,0,2,1,2,400000.0,400000.0,0.0,2,0,2,2,2,0.0
3,20267,1.0,Y,7.0,6.0,8951.0,3.0,25700.0,1600000.0,1600000.0,0.0,0,5,2,10,74755070.0,74734000.0,0.0,43,805,15,5,23,0.0
4,2778,1.0,N,0.0,6.0,4680.0,8.0,12423.0,260000.0,260000.0,0.0,0,4,3,7,6070000.0,6070000.0,0.0,10,144,6,2,6,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14672,14012,1.0,N,1.0,6.0,4284.0,6.0,12696.0,330000.0,330000.0,0.0,0,1,2,1,15330000.0,15330000.0,0.0,2,9,1,1,1,0.0
14673,15203,1.0,N,5.0,6.0,8510.0,3.0,9182.0,125613.0,125613.0,0.0,0,1,2,1,159513.0,159513.0,0.0,2,4,1,1,1,0.0
14674,16048,1.0,N,1.0,1.0,4016.0,1.0,14029.0,80000.0,80000.0,0.0,0,1,2,1,300000.0,300000.0,0.0,2,8,1,1,1,0.0
14675,2924,1.0,N,14.0,6.0,7228.0,2.0,8671.0,90000.0,90000.0,0.0,0,1,1,1,90000.0,90000.0,0.0,1,12,1,1,1,0.0


In [338]:
merged_data.to_csv('merged_data.csv', encoding = 'utf-8')#utf-8방식 csv파일 전환

In [339]:
files.download('merged_data.csv')#구글 코랩 내장 다운로드 함수

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>